In [8]:
# =========================
# STEP 1 — Setup & Load
# =========================
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 180)

# --- Load ---
DATA_PATH = "accepted_2007_to_2018Q4.csv"
df_raw = pd.read_csv(DATA_PATH)

print("Raw shape:", df_raw.shape)
print(df_raw.head(3))
print(df_raw.info())

# =========================
# STEP 1.1 — Ensure binary target exists (1=default, 0=paid)
# =========================
df = df_raw.copy()

if "loan_status" not in df.columns:
    raise ValueError("Expected 'loan_status' in the dataset.")

# Map textual labels -> {Fully Paid:0, Charged Off:1}; otherwise coerce
if df["loan_status"].dtype == "object":
    df["loan_status"] = df["loan_status"].map({"Fully Paid": 0, "Charged Off": 1})

df["loan_status"] = pd.to_numeric(df["loan_status"], errors="coerce").astype("Int64")
before = len(df)
df = df[df["loan_status"].isin([0, 1])].copy()
after = len(df)
if after < before:
    print(f"Dropped {before-after} rows with invalid loan_status values.")
df["loan_status"] = df["loan_status"].astype("int8")

print("\nTarget distribution (%):")
print((df["loan_status"].value_counts(normalize=True)*100).round(2).rename({1:"default(1)", 0:"fully_paid(0)"}))
print("Shape after target cleanup:", df.shape)

# Light cleaning (cheap, no feature eng)
if "term" in df.columns and df["term"].dtype == "object":
    df["term"] = df["term"].astype(str).str.extract(r"(\d+)").astype(float)

if "int_rate" in df.columns and df["int_rate"].dtype == "object":
    df["int_rate"] = pd.to_numeric(df["int_rate"].astype(str).str.replace("%", "", regex=False), errors="coerce")

drop_optional = [c for c in ["grade"] if c in df.columns]
if drop_optional:
    print("Dropping optional redundant columns:", drop_optional)
    df = df.drop(columns=drop_optional)

# =========================
# STEP 2 — Drop very-missing columns (row imputation happens later in pipeline)
# =========================
MISS_THRESH = 0.40  # drop columns with >40% missing
miss_ratio = df.isna().mean().sort_values(ascending=False)
to_drop = miss_ratio[miss_ratio > MISS_THRESH].index.tolist()
to_drop = [c for c in to_drop if c != "loan_status"]  # never drop target

if to_drop:
    print(f"\nColumns dropped for high missingness (> {int(MISS_THRESH*100)}%): {to_drop}")
    df = df.drop(columns=to_drop)

print("\nTop missing % after column drop:\n", (df.isna().mean().sort_values(ascending=False).head(15)*100).round(2))

# =========================
# STEP 3 — Optional row downsampling to 500k (stratified)
# =========================
RSEED = 42
TARGET_ROWS = 500_000
if len(df) > TARGET_ROWS:
    frac_keep = TARGET_ROWS / len(df)
    print(f"\nDownsampling rows to ~{TARGET_ROWS} (keep fraction ≈ {frac_keep:.3f}) with stratification...")
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1-frac_keep, random_state=RSEED)
    y_tmp = df["loan_status"]
    for keep_idx, _ in sss.split(df, y_tmp):
        df = df.iloc[keep_idx].copy()
    print("New shape after downsampling:", df.shape)
    print("New target %:", (df["loan_status"].value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 4 — Split
# =========================
y_full = df["loan_status"].astype("int8")
X_full = df.drop(columns=["loan_status"])

X_train, X_temp, y_train, y_temp = train_test_split(
    X_full, y_full, test_size=0.30, stratify=y_full, random_state=RSEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RSEED
)

print("\nSizes ->", X_train.shape, X_val.shape, X_test.shape)
print("Class % (train):", (y_train.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (val)  :", (y_val.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (test) :", (y_test.value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 5 — Preprocessing (Impute + cast categoricals -> str + sparse OHE with rare-category capping)
# =========================
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config

set_config(transform_output="default")  # keep default outputs

# Identify columns by dtype on X_full (after drops)
num_cols = [c for c in X_full.columns if pd.api.types.is_numeric_dtype(X_full[c])]
cat_cols = [c for c in X_full.columns if c not in num_cols]  # treat everything else as categorical

print(f"\nDetected {len(num_cols)} numeric and {len(cat_cols)} categorical columns.")

numeric_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True)),
    ("to_float32", FunctionTransformer(lambda X: X.astype(np.float32)))
])

# Keep sparse; bucket rare categories; cast to string after impute
categorical_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(lambda X: X.astype(str))),
    ("ohe", OneHotEncoder(
        handle_unknown="infrequent_if_exist",
        min_frequency=0.005,             # keep cats >=0.5% freq; rest -> "infrequent"
        sparse_output=True,              # CRITICAL: keep sparse
        dtype=np.float32,
        drop="if_binary"
    ))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=0.1                 # favor sparse stacking
)

# =========================
# STEP 6 — Models (sparse-friendly only)
# =========================
from sklearn.linear_model import LogisticRegression
try:
    from xgboost import XGBClassifier
except Exception as e:
    raise RuntimeError("xgboost not installed. Run: pip install xgboost") from e

scale_pos_weight = max(1.0, (y_train == 0).sum() / max(1, (y_train == 1).sum()))
print("scale_pos_weight (train):", round(scale_pos_weight, 3))

logit_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(
        solver="saga",
        penalty="l2",
        C=1.0,
        max_iter=300,
        class_weight="balanced",
        random_state=RSEED,
        n_jobs=-1
    ))
])

xgb_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", XGBClassifier(
        n_estimators=250,
        max_depth=6,
        learning_rate=0.08,
        subsample=0.9,
        colsample_bytree=0.8,
        min_child_weight=5,
        reg_lambda=1.0,
        objective="binary:logistic",
        scale_pos_weight=scale_pos_weight,
        tree_method="hist",
        random_state=RSEED,
        n_jobs=-1,
        max_bin=256
    ))
])

models = {
    "LogReg": logit_pipe,
    "XGBoost": xgb_pipe
}
print("Models ready:", list(models.keys()))

# =========================
# STEP 7 — Evaluation (Precision @ target recall)
# =========================
RECALL_TARGETS = (0.80, 0.85, 0.90)

def pick_threshold_at_min_recall(y_true, p1, min_recall):
    """Pick threshold that achieves recall >= min_recall with max precision (on validation)."""
    prec, rec, ths = precision_recall_curve(y_true, p1, pos_label=1)
    idxs = np.where(rec >= min_recall)[0]
    if len(idxs) == 0:
        return None, None, None
    best_i = idxs[np.argmax(prec[idxs])]
    thr = ths[min(best_i, len(ths)-1)]   # ths has len = len(prec)-1
    return float(thr), float(prec[best_i]), float(rec[best_i])

def eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test, recall_targets=RECALL_TARGETS):
    pipe.fit(X_train, y_train)

    # Validation for threshold selection
    p_val = pipe.predict_proba(X_val)[:, 1]
    ap = average_precision_score(y_val, p_val)
    print(f"\n=== {name} ===  AP(Val)={ap:.4f}")

    results = {}
    for tgt in recall_targets:
        thr, p_at, r_at = pick_threshold_at_min_recall(y_val, p_val, tgt)
        if thr is None:
            print(f"  recall≥{tgt:.2f}: not reachable.")
            continue

        p_test = pipe.predict_proba(X_test)[:, 1]
        yhat = (p_test >= thr).astype("int8")
        cm = confusion_matrix(y_test, yhat, labels=[1, 0])  # rows: actual 1,0 ; cols: pred 1,0
        TP, FN, FP, TN = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
        prec_test = TP/(TP+FP) if (TP+FP) > 0 else 0.0
        rec_test  = TP/(TP+FN) if (TP+FN) > 0 else 0.0

        print(f"  recall≥{tgt:.2f}: thr={thr:.4f} | Val P={p_at:.3f}, R={r_at:.3f} | Test precision={prec_test:.3f}, recall={rec_test:.3f}")
        results[tgt] = {"thr": thr, "precision_test": prec_test, "recall_test": rec_test, "cm": cm}
    return ap, results

all_results = {}
for name, pipe in models.items():
    ap, res = eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test)
    all_results[name] = {"AP_val": ap, "by_recall": res}

print("\nSummary (precision at target recalls on TEST):")
for name, info in all_results.items():
    row = [name]
    for tgt in RECALL_TARGETS:
        if tgt in info["by_recall"]:
            row.append(f"R≥{tgt:.2f}: P={info['by_recall'][tgt]['precision_test']:.3f}")
        else:
            row.append(f"R≥{tgt:.2f}: NA")
    print(" | ".join(row))


Raw shape: (2260701, 151)
         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  installment grade sub_grade     emp_title emp_length home_ownership  annual_inc  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months     13.99       123.03     C        C4       leadman  10+ years       MORTGAGE     55000.0   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months     11.99       820.28     C        C1      Engineer  10+ years       MORTGAGE     65000.0   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months     10.78       432.66     B        B4  truck driver  10+ years       MORTGAGE     63000.0   

  verification_status   issue_d loan_status pymnt_plan                                                url desc             purpose               title zip_code addr_state    dti  \
0        Not Verified  Dec-2015  Fully Paid          n  https://lendingclub.com/browse/loanDetail.acti...  NaN  

Dropped 915391 rows with invalid loan_status values.

Target distribution (%):
loan_status
fully_paid(0)    80.04
default(1)       19.96
Name: proportion, dtype: float64
Shape after target cleanup: (1345310, 151)
Dropping optional redundant columns: ['grade']

Columns dropped for high missingness (> 40%): ['member_id', 'next_pymnt_d', 'orig_projected_additional_accrued_interest', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount', 'hardship_type', 'hardship_status', 'hardship_amount', 'deferral_term', 'hardship_end_date', 'hardship_dpd', 'hardship_loan_status', 'hardship_length', 'payment_plan_start_date', 'hardship_reason', 'hardship_start_date', 'sec_app_mths_since_last_major_derog', 'sec_app_revol_util', 'revol_bal_joint', 'sec_app_mort_acc', 'sec_app_inq_last_6mths', 'sec_app_earliest_cr_line', 'sec_app_open_act_il', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'sec_app_num_rev_accts', 'sec_app_collections_12_mths_ex_med', 'sec_app_chargeoff_within_12_mths', '

In [9]:
# =========================
# STEP 1 — Setup & Load
# =========================
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 180)

# --- Load ---
DATA_PATH = "accepted_2007_to_2018Q4.csv"
df_raw = pd.read_csv(DATA_PATH)

print("Raw shape:", df_raw.shape)
print(df_raw.head(3))
print(df_raw.info())

# =========================
# STEP 1.1 — Ensure binary target exists (1=default, 0=paid)
# =========================
df = df_raw.copy()

if "loan_status" not in df.columns:
    raise ValueError("Expected 'loan_status' in the dataset.")

# Map textual labels -> {Fully Paid:0, Charged Off:1}; otherwise coerce
if df["loan_status"].dtype == "object":
    df["loan_status"] = df["loan_status"].map({"Fully Paid": 0, "Charged Off": 1})

df["loan_status"] = pd.to_numeric(df["loan_status"], errors="coerce").astype("Int64")
before = len(df)
df = df[df["loan_status"].isin([0, 1])].copy()
after = len(df)
if after < before:
    print(f"Dropped {before-after} rows with invalid loan_status values.")
df["loan_status"] = df["loan_status"].astype("int8")

print("\nTarget distribution (%):")
print((df["loan_status"].value_counts(normalize=True)*100).round(2).rename({1:"default(1)", 0:"fully_paid(0)"}))
print("Shape after target cleanup:", df.shape)

# Light cleaning
if "term" in df.columns and df["term"].dtype == "object":
    df["term"] = df["term"].astype(str).str.extract(r"(\d+)").astype(float)

if "int_rate" in df.columns and df["int_rate"].dtype == "object":
    df["int_rate"] = pd.to_numeric(df["int_rate"].astype(str).str.replace("%", "", regex=False), errors="coerce")

drop_optional = [c for c in ["grade"] if c in df.columns]
if drop_optional:
    print("Dropping optional redundant columns:", drop_optional)
    df = df.drop(columns=drop_optional)

# =========================
# STEP 2 — Drop leaky columns (post-origination info)
# =========================
LEAKY_COLS = [
    # outcome / payment history
    "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv",
    "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee",
    "last_pymnt_d", "last_pymnt_amnt", "last_credit_pull_d", "next_pymnt_d",

    # hardship / settlement / plans
    "hardship_flag", "hardship_type", "hardship_reason", "hardship_status",
    "hardship_amount", "hardship_start_date", "hardship_end_date",
    "hardship_length", "hardship_dpd", "hardship_loan_status",
    "payment_plan_start_date",
    "debt_settlement_flag", "debt_settlement_flag_date",
    "settlement_status", "settlement_date", "settlement_amount",
    "settlement_percentage", "settlement_term",

    # identifiers / flags that don’t belong
    "url", "pymnt_plan"
]

df = df.drop(columns=[c for c in LEAKY_COLS if c in df.columns])
print(f"\nDropped {len([c for c in LEAKY_COLS if c in df.columns])} leaky columns.")

# =========================
# STEP 3 — Drop very-missing columns (>40% NaN)
# =========================
MISS_THRESH = 0.40
miss_ratio = df.isna().mean().sort_values(ascending=False)
to_drop = miss_ratio[miss_ratio > MISS_THRESH].index.tolist()
to_drop = [c for c in to_drop if c != "loan_status"]

if to_drop:
    print(f"\nColumns dropped for high missingness (> {int(MISS_THRESH*100)}%): {to_drop}")
    df = df.drop(columns=to_drop)

print("\nTop missing % after column drop:\n", (df.isna().mean().sort_values(ascending=False).head(15)*100).round(2))

# =========================
# STEP 4 — Downsample to ~500k rows (stratified)
# =========================
RSEED = 42
TARGET_ROWS = 500_000
if len(df) > TARGET_ROWS:
    frac_keep = TARGET_ROWS / len(df)
    print(f"\nDownsampling rows to ~{TARGET_ROWS} (keep fraction ≈ {frac_keep:.3f})...")
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1-frac_keep, random_state=RSEED)
    y_tmp = df["loan_status"]
    for keep_idx, _ in sss.split(df, y_tmp):
        df = df.iloc[keep_idx].copy()
    print("New shape after downsampling:", df.shape)
    print("New target %:", (df["loan_status"].value_counts(normalize=True)*100).round(2).to_dict())

    
# ========================
# STEP 5 — Split
# =========================
y_full = df["loan_status"].astype("int8")
X_full = df.drop(columns=["loan_status"])

X_train, X_temp, y_train, y_temp = train_test_split(
    X_full, y_full, test_size=0.30, stratify=y_full, random_state=RSEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RSEED
)

print("\nSizes ->", X_train.shape, X_val.shape, X_test.shape)
print("Class % (train):", (y_train.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (val)  :", (y_val.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (test) :", (y_test.value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 6 — Preprocessing (Impute + cast categoricals -> str + sparse OHE with rare-category capping)
# =========================
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config

set_config(transform_output="default")  # keep default outputs

# Identify columns by dtype on X_full (after drops)
num_cols = [c for c in X_full.columns if pd.api.types.is_numeric_dtype(X_full[c])]
cat_cols = [c for c in X_full.columns if c not in num_cols]  # treat everything else as categorical

print(f"\nDetected {len(num_cols)} numeric and {len(cat_cols)} categorical columns.")

numeric_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True)),
    ("to_float32", FunctionTransformer(lambda X: X.astype(np.float32)))
])

# Keep sparse; bucket rare categories; cast to string after impute
categorical_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(lambda X: X.astype(str))),
    ("ohe", OneHotEncoder(
        handle_unknown="infrequent_if_exist",
        min_frequency=0.005,             # keep cats >=0.5% freq; rest -> "infrequent"
        sparse_output=True,              # CRITICAL: keep sparse
        dtype=np.float32,
        drop="if_binary"
    ))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=0.1                 # favor sparse stacking
)

# =========================
# STEP 7 — Models (sparse-friendly only)
# =========================
from sklearn.linear_model import LogisticRegression
try:
    from xgboost import XGBClassifier
except Exception as e:
    raise RuntimeError("xgboost not installed. Run: pip install xgboost") from e

scale_pos_weight = max(1.0, (y_train == 0).sum() / max(1, (y_train == 1).sum()))
print("scale_pos_weight (train):", round(scale_pos_weight, 3))

logit_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(
        solver="saga",
        penalty="l2",
        C=1.0,
        max_iter=300,
        class_weight="balanced",
        random_state=RSEED,
        n_jobs=-1
    ))
])

xgb_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", XGBClassifier(
        n_estimators=250,
        max_depth=6,
        learning_rate=0.08,
        subsample=0.9,
        colsample_bytree=0.8,
        min_child_weight=5,
        reg_lambda=1.0,
        objective="binary:logistic",
        scale_pos_weight=scale_pos_weight,
        tree_method="hist",
        random_state=RSEED,
        n_jobs=-1,
        max_bin=256
    ))
])

models = {
    "LogReg": logit_pipe,
    "XGBoost": xgb_pipe
}
print("Models ready:", list(models.keys()))

# =========================
# STEP 8 — Evaluation (Precision @ target recall)
# =========================
RECALL_TARGETS = (0.80, 0.85, 0.90)

def pick_threshold_at_min_recall(y_true, p1, min_recall):
    """Pick threshold that achieves recall >= min_recall with max precision (on validation)."""
    prec, rec, ths = precision_recall_curve(y_true, p1, pos_label=1)
    idxs = np.where(rec >= min_recall)[0]
    if len(idxs) == 0:
        return None, None, None
    best_i = idxs[np.argmax(prec[idxs])]
    thr = ths[min(best_i, len(ths)-1)]   # ths has len = len(prec)-1
    return float(thr), float(prec[best_i]), float(rec[best_i])

def eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test, recall_targets=RECALL_TARGETS):
    pipe.fit(X_train, y_train)

    # Validation for threshold selection
    p_val = pipe.predict_proba(X_val)[:, 1]
    ap = average_precision_score(y_val, p_val)
    print(f"\n=== {name} ===  AP(Val)={ap:.4f}")

    results = {}
    for tgt in recall_targets:
        thr, p_at, r_at = pick_threshold_at_min_recall(y_val, p_val, tgt)
        if thr is None:
            print(f"  recall≥{tgt:.2f}: not reachable.")
            continue

        p_test = pipe.predict_proba(X_test)[:, 1]
        yhat = (p_test >= thr).astype("int8")
        cm = confusion_matrix(y_test, yhat, labels=[1, 0])  # rows: actual 1,0 ; cols: pred 1,0
        TP, FN, FP, TN = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
        prec_test = TP/(TP+FP) if (TP+FP) > 0 else 0.0
        rec_test  = TP/(TP+FN) if (TP+FN) > 0 else 0.0

        print(f"  recall≥{tgt:.2f}: thr={thr:.4f} | Val P={p_at:.3f}, R={r_at:.3f} | Test precision={prec_test:.3f}, recall={rec_test:.3f}")
        results[tgt] = {"thr": thr, "precision_test": prec_test, "recall_test": rec_test, "cm": cm}
    return ap, results

all_results = {}
for name, pipe in models.items():
    ap, res = eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test)
    all_results[name] = {"AP_val": ap, "by_recall": res}

print("\nSummary (precision at target recalls on TEST):")
for name, info in all_results.items():
    row = [name]
    for tgt in RECALL_TARGETS:
        if tgt in info["by_recall"]:
            row.append(f"R≥{tgt:.2f}: P={info['by_recall'][tgt]['precision_test']:.3f}")
        else:
            row.append(f"R≥{tgt:.2f}: NA")
    print(" | ".join(row))
    


Raw shape: (2260701, 151)
         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  installment grade sub_grade     emp_title emp_length home_ownership  annual_inc  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months     13.99       123.03     C        C4       leadman  10+ years       MORTGAGE     55000.0   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months     11.99       820.28     C        C1      Engineer  10+ years       MORTGAGE     65000.0   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months     10.78       432.66     B        B4  truck driver  10+ years       MORTGAGE     63000.0   

  verification_status   issue_d loan_status pymnt_plan                                                url desc             purpose               title zip_code addr_state    dti  \
0        Not Verified  Dec-2015  Fully Paid          n  https://lendingclub.com/browse/loanDetail.acti...  NaN  

Dropped 915391 rows with invalid loan_status values.

Target distribution (%):
loan_status
fully_paid(0)    80.04
default(1)       19.96
Name: proportion, dtype: float64
Shape after target cleanup: (1345310, 151)
Dropping optional redundant columns: ['grade']

Dropped 0 leaky columns.

Columns dropped for high missingness (> 40%): ['member_id', 'orig_projected_additional_accrued_interest', 'hardship_last_payment_amount', 'deferral_term', 'hardship_payoff_balance_amount', 'sec_app_mths_since_last_major_derog', 'sec_app_revol_util', 'revol_bal_joint', 'sec_app_fico_range_high', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_fico_range_low', 'sec_app_num_rev_accts', 'sec_app_inq_last_6mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_earliest_cr_line', 'sec_app_chargeoff_within_12_mths', 'verification_status_joint', 'dti_joint', 'annual_inc_joint', 'desc', 'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'mths_since_recent_r

In [10]:
# =========================
# FEATURE IMPORTANCE: LogReg + XGBoost
# =========================
import numpy as np

def get_feature_names(preprocessor):
    """Extract feature names from ColumnTransformer after OHE + scaling."""
    num_features = preprocessor.named_transformers_["num"]["impute"].feature_names_in_
    cat_features = preprocessor.named_transformers_["cat"]["ohe"].get_feature_names_out(preprocessor.transformers_[1][2])
    return np.concatenate([num_features, cat_features])

# 1) Logistic Regression
logreg_model = models["LogReg"].named_steps["model"]
feature_names = get_feature_names(models["LogReg"].named_steps["preprocess"])
logreg_coefs = pd.Series(logreg_model.coef_[0], index=feature_names).sort_values(key=abs, ascending=False)

print("\nTop 10 LogReg features (by absolute coefficient):")
print(logreg_coefs.head(10))

# 2) XGBoost
xgb_model = models["XGBoost"].named_steps["model"]
xgb_importances = pd.Series(xgb_model.feature_importances_, index=feature_names).sort_values(ascending=False)

print("\nTop 10 XGBoost features (by importance):")
print(xgb_importances.head(10))



Top 10 LogReg features (by absolute coefficient):
last_fico_range_high           -1.847535
last_fico_range_low            -0.980509
emp_title_infrequent_sklearn   -0.300951
term                            0.259335
id_infrequent_sklearn          -0.237214
emp_title_Teacher               0.168807
dti                             0.165097
int_rate                        0.144501
mo_sin_old_rev_tl_op            0.128806
home_ownership_MORTGAGE        -0.125083
dtype: float32

Top 10 XGBoost features (by importance):
last_fico_range_high            0.189705
last_fico_range_low             0.095982
term                            0.032643
emp_title_Teacher               0.011906
title_infrequent_sklearn        0.011111
funded_amnt                     0.008602
emp_title_infrequent_sklearn    0.006299
application_type_Joint App      0.005438
issue_d_infrequent_sklearn      0.005434
loan_amnt                       0.005155
dtype: float32


In [11]:
# =========================
# CLEAN TRAINING PIPELINE (no leakage)
# =========================
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 180)

# --- Load ---
DATA_PATH = "accepted_2007_to_2018Q4.csv"
df_raw = pd.read_csv(DATA_PATH)

print("Raw shape:", df_raw.shape)
print(df_raw.head(3))
print(df_raw.info())

# =========================
# STEP 1 — Target cleanup (1=default, 0=paid)
# =========================
df = df_raw.copy()

if "loan_status" not in df.columns:
    raise ValueError("Expected 'loan_status' in the dataset.")

# Map textual labels -> {Fully Paid:0, Charged Off:1}; otherwise coerce
if df["loan_status"].dtype == "object":
    df["loan_status"] = df["loan_status"].map({"Fully Paid": 0, "Charged Off": 1})

df["loan_status"] = pd.to_numeric(df["loan_status"], errors="coerce").astype("Int64")
before = len(df)
df = df[df["loan_status"].isin([0, 1])].copy()
after = len(df)
if after < before:
    print(f"Dropped {before-after} rows with invalid loan_status values.")
df["loan_status"] = df["loan_status"].astype("int8")

print("\nTarget distribution (%):")
print((df["loan_status"].value_counts(normalize=True)*100).round(2).rename({1:"default(1)", 0:"fully_paid(0)"}))
print("Shape after target cleanup:", df.shape)

# =========================
# STEP 2 — Light cleaning
# =========================
# "36 months" -> 36
if "term" in df.columns and df["term"].dtype == "object":
    df["term"] = df["term"].astype(str).str.extract(r"(\d+)").astype(float)

# "13.99%" -> 13.99
if "int_rate" in df.columns and df["int_rate"].dtype == "object":
    df["int_rate"] = pd.to_numeric(df["int_rate"].astype(str).str.replace("%", "", regex=False), errors="coerce")

# drop redundant grade (sub_grade kept)
drop_optional = [c for c in ["grade"] if c in df.columns]
if drop_optional:
    print("Dropping optional redundant columns:", drop_optional)
    df = df.drop(columns=drop_optional)

# =========================
# STEP 3 — Drop leaky columns (post-origination / identifiers)
# =========================
LEAKY_COLS = [
    # outcome / payment history (definitely post-origination)
    "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv",
    "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee",
    "last_pymnt_d", "last_pymnt_amnt", "last_credit_pull_d", "next_pymnt_d",

    # hardship / settlement / plans
    "hardship_flag", "hardship_type", "hardship_reason", "hardship_status",
    "hardship_amount", "hardship_start_date", "hardship_end_date",
    "hardship_length", "hardship_dpd", "hardship_loan_status",
    "payment_plan_start_date",
    "debt_settlement_flag", "debt_settlement_flag_date",
    "settlement_status", "settlement_date", "settlement_amount",
    "settlement_percentage", "settlement_term",

    # NEWLY ADDED (we saw them dominating importances)
    "last_fico_range_high", "last_fico_range_low",

    # identifiers we don't want to learn from
    "url", "pymnt_plan", "id"
]
present_leaky = [c for c in LEAKY_COLS if c in df.columns]
df = df.drop(columns=present_leaky)
print(f"\nDropped {len(present_leaky)} leaky columns: {present_leaky}")

# =========================
# STEP 4 — Drop very-missing columns (>40% NaN)
# =========================
MISS_THRESH = 0.40
miss_ratio = df.isna().mean().sort_values(ascending=False)
to_drop = miss_ratio[miss_ratio > MISS_THRESH].index.tolist()
to_drop = [c for c in to_drop if c != "loan_status"]

if to_drop:
    print(f"\nColumns dropped for high missingness (> {int(MISS_THRESH*100)}%): {to_drop}")
    df = df.drop(columns=to_drop)

print("\nTop missing % after column drop:\n", (df.isna().mean().sort_values(ascending=False).head(15)*100).round(2))

# =========================
# STEP 5 — Downsample to ~500k rows (stratified)
# =========================
RSEED = 42
TARGET_ROWS = 500_000
if len(df) > TARGET_ROWS:
    frac_keep = TARGET_ROWS / len(df)
    print(f"\nDownsampling rows to ~{TARGET_ROWS} (keep fraction ≈ {frac_keep:.3f})...")
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1-frac_keep, random_state=RSEED)
    y_tmp = df["loan_status"]
    for keep_idx, _ in sss.split(df, y_tmp):
        df = df.iloc[keep_idx].copy()
    print("New shape after downsampling:", df.shape)
    print("New target %:", (df["loan_status"].value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 6 — Split
# =========================
y_full = df["loan_status"].astype("int8")
X_full = df.drop(columns=["loan_status"])

X_train, X_temp, y_train, y_temp = train_test_split(
    X_full, y_full, test_size=0.30, stratify=y_full, random_state=RSEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RSEED
)

print("\nSizes ->", X_train.shape, X_val.shape, X_test.shape)
print("Class % (train):", (y_train.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (val)  :", (y_val.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (test) :", (y_test.value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 7 — Preprocessing (sparse OHE + rare bucket)
# =========================
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config

set_config(transform_output="default")

# Identify columns by dtype on X_full (after drops)
num_cols = [c for c in X_full.columns if pd.api.types.is_numeric_dtype(X_full[c])]
cat_cols = [c for c in X_full.columns if c not in num_cols]  # treat everything else as categorical

print(f"\nDetected {len(num_cols)} numeric and {len(cat_cols)} categorical columns.")

numeric_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True)),
    ("to_float32", FunctionTransformer(lambda X: X.astype(np.float32)))
])

categorical_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(lambda X: X.astype(str))),
    ("ohe", OneHotEncoder(
        handle_unknown="infrequent_if_exist",
        min_frequency=0.005,        # keep cats >=0.5% freq; others -> 'infrequent'
        sparse_output=True,
        dtype=np.float32,
        drop="if_binary"
    ))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=0.1
)

# =========================
# STEP 8 — Models (sparse-friendly)
# =========================
from sklearn.linear_model import LogisticRegression
try:
    from xgboost import XGBClassifier
except Exception as e:
    raise RuntimeError("xgboost not installed. Run: pip install xgboost") from e

scale_pos_weight = max(1.0, (y_train == 0).sum() / max(1, (y_train == 1).sum()))
print("scale_pos_weight (train):", round(scale_pos_weight, 3))

logit_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(
        solver="saga",
        penalty="l2",
        C=1.0,
        max_iter=300,
        class_weight="balanced",
        random_state=RSEED,
        n_jobs=-1
    ))
])

xgb_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", XGBClassifier(
        n_estimators=250,
        max_depth=6,
        learning_rate=0.08,
        subsample=0.9,
        colsample_bytree=0.8,
        min_child_weight=5,
        reg_lambda=1.0,
        objective="binary:logistic",
        scale_pos_weight=scale_pos_weight,
        tree_method="hist",
        random_state=RSEED,
        n_jobs=-1,
        max_bin=256
    ))
])

models = {
    "LogReg": logit_pipe,
    "XGBoost": xgb_pipe
}
print("Models ready:", list(models.keys()))

# =========================
# STEP 9 — Evaluation (Precision @ target recall)
# =========================
RECALL_TARGETS = (0.80, 0.85, 0.90)

def pick_threshold_at_min_recall(y_true, p1, min_recall):
    """Pick threshold that achieves recall >= min_recall with max precision (on validation)."""
    prec, rec, ths = precision_recall_curve(y_true, p1, pos_label=1)
    idxs = np.where(rec >= min_recall)[0]
    if len(idxs) == 0:
        return None, None, None
    best_i = idxs[np.argmax(prec[idxs])]
    thr = ths[min(best_i, len(ths)-1)]   # ths has len = len(prec)-1
    return float(thr), float(prec[best_i]), float(rec[best_i])

def eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test, recall_targets=RECALL_TARGETS):
    pipe.fit(X_train, y_train)

    # Validation for threshold selection
    p_val = pipe.predict_proba(X_val)[:, 1]
    ap = average_precision_score(y_val, p_val)
    print(f"\n=== {name} ===  AP(Val)={ap:.4f}")

    results = {}
    for tgt in recall_targets:
        thr, p_at, r_at = pick_threshold_at_min_recall(y_val, p_val, tgt)
        if thr is None:
            print(f"  recall≥{tgt:.2f}: not reachable.")
            continue

        p_test = pipe.predict_proba(X_test)[:, 1]
        yhat = (p_test >= thr).astype("int8")
        cm = confusion_matrix(y_test, yhat, labels=[1, 0])  # rows: actual 1,0 ; cols: pred 1,0
        TP, FN, FP, TN = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
        prec_test = TP/(TP+FP) if (TP+FP) > 0 else 0.0
        rec_test  = TP/(TP+FN) if (TP+FN) > 0 else 0.0

        print(f"  recall≥{tgt:.2f}: thr={thr:.4f} | Val P={p_at:.3f}, R={r_at:.3f} | Test precision={prec_test:.3f}, recall={rec_test:.3f}")
        results[tgt] = {"thr": thr, "precision_test": prec_test, "recall_test": rec_test, "cm": cm}
    return ap, results

all_results = {}
for name, pipe in models.items():
    ap, res = eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test)
    all_results[name] = {"AP_val": ap, "by_recall": res}

print("\nSummary (precision at target recalls on TEST):")
for name, info in all_results.items():
    row = [name]
    for tgt in RECALL_TARGETS:
        if tgt in info["by_recall"]:
            row.append(f"R≥{tgt:.2f}: P={info['by_recall'][tgt]['precision_test']:.3f}")
        else:
            row.append(f"R≥{tgt:.2f}: NA")
    print(" | ".join(row))

# =========================
# STEP 10 — Feature importance (optional sanity-check)
# =========================
# helper to get final feature names from the preprocessor
def get_ohe_feature_names(preprocessor, num_cols, cat_cols):
    num_out = np.array(num_cols, dtype=object)
    ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
    cat_out = ohe.get_feature_names_out(cat_cols)
    return np.concatenate([num_out, cat_out])

# Logistic Regression: top coefficients
pre = logit_pipe.named_steps["preprocess"]
feature_names = get_ohe_feature_names(pre, num_cols, cat_cols)
log_coefs = pd.Series(logit_pipe.named_steps["model"].coef_.ravel(), index=feature_names)

print("\nTop 15 LogReg features (by absolute coefficient):")
print(log_coefs.reindex(log_coefs.abs().sort_values(ascending=False).head(15).index))

# XGBoost: top importance
pre_x = xgb_pipe.named_steps["preprocess"]
feature_names_x = get_ohe_feature_names(pre_x, num_cols, cat_cols)
xgb_model = xgb_pipe.named_steps["model"]
xgb_import = pd.Series(xgb_model.feature_importances_, index=feature_names_x).sort_values(ascending=False)

print("\nTop 15 XGBoost features (by importance):")
print(xgb_import.head(15))


Raw shape: (2260701, 151)
         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  installment grade sub_grade     emp_title emp_length home_ownership  annual_inc  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months     13.99       123.03     C        C4       leadman  10+ years       MORTGAGE     55000.0   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months     11.99       820.28     C        C1      Engineer  10+ years       MORTGAGE     65000.0   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months     10.78       432.66     B        B4  truck driver  10+ years       MORTGAGE     63000.0   

  verification_status   issue_d loan_status pymnt_plan                                                url desc             purpose               title zip_code addr_state    dti  \
0        Not Verified  Dec-2015  Fully Paid          n  https://lendingclub.com/browse/loanDetail.acti...  NaN  

Dropped 915391 rows with invalid loan_status values.

Target distribution (%):
loan_status
fully_paid(0)    80.04
default(1)       19.96
Name: proportion, dtype: float64
Shape after target cleanup: (1345310, 151)
Dropping optional redundant columns: ['grade']

Dropped 36 leaky columns: ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'next_pymnt_d', 'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'payment_plan_start_date', 'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term', 'last_fico_range_high', 'last_fico_range_low', 'url', 'pymnt_plan', 'id']

Columns dropped for hi

In [12]:
# =========================
# FEATURE IMPORTANCE: LogReg + XGBoost
# =========================
import numpy as np

def get_feature_names(preprocessor):
    """Extract feature names from ColumnTransformer after OHE + scaling."""
    num_features = preprocessor.named_transformers_["num"]["impute"].feature_names_in_
    cat_features = preprocessor.named_transformers_["cat"]["ohe"].get_feature_names_out(preprocessor.transformers_[1][2])
    return np.concatenate([num_features, cat_features])

# 1) Logistic Regression
logreg_model = models["LogReg"].named_steps["model"]
feature_names = get_feature_names(models["LogReg"].named_steps["preprocess"])
logreg_coefs = pd.Series(logreg_model.coef_[0], index=feature_names).sort_values(key=abs, ascending=False)

print("\nTop 10 LogReg features (by absolute coefficient):")
print(logreg_coefs.head(10))

# 2) XGBoost
xgb_model = models["XGBoost"].named_steps["model"]
xgb_importances = pd.Series(xgb_model.feature_importances_, index=feature_names).sort_values(ascending=False)

print("\nTop 10 XGBoost features (by importance):")
print(xgb_importances.head(10))



Top 10 LogReg features (by absolute coefficient):
int_rate                        0.335715
term                            0.254939
emp_title_Teacher               0.174230
emp_title_infrequent_sklearn   -0.173747
dti                             0.166887
title_infrequent_sklearn       -0.138980
acc_open_past_24mths            0.125175
issue_d_infrequent_sklearn     -0.125062
issue_d_Apr-2016                0.125004
purpose_small_business          0.125002
dtype: float32

Top 10 XGBoost features (by importance):
int_rate                            0.063232
term                                0.026783
sub_grade_A1                        0.022705
sub_grade_B3                        0.013335
sub_grade_B4                        0.012699
title_infrequent_sklearn            0.012664
sub_grade_A3                        0.012356
home_ownership_RENT                 0.010203
emp_title_infrequent_sklearn        0.009740
verification_status_Not Verified    0.009040
dtype: float32


In [13]:
# =========================
# CLEAN TRAINING PIPELINE + DecisionTree & RandomForest
# =========================
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 180)

# --- Load ---
DATA_PATH = "accepted_2007_to_2018Q4.csv"
df_raw = pd.read_csv(DATA_PATH)

print("Raw shape:", df_raw.shape)
print(df_raw.head(3))
print(df_raw.info())

# =========================
# STEP 1 — Target cleanup (1=default, 0=paid)
# =========================
df = df_raw.copy()

if "loan_status" not in df.columns:
    raise ValueError("Expected 'loan_status' in the dataset.")

# Map textual labels -> {Fully Paid:0, Charged Off:1}; otherwise coerce
if df["loan_status"].dtype == "object":
    df["loan_status"] = df["loan_status"].map({"Fully Paid": 0, "Charged Off": 1})

df["loan_status"] = pd.to_numeric(df["loan_status"], errors="coerce").astype("Int64")
before = len(df)
df = df[df["loan_status"].isin([0, 1])].copy()
after = len(df)
if after < before:
    print(f"Dropped {before-after} rows with invalid loan_status values.")
df["loan_status"] = df["loan_status"].astype("int8")

print("\nTarget distribution (%):")
print((df["loan_status"].value_counts(normalize=True)*100).round(2).rename({1:"default(1)", 0:"fully_paid(0)"}))
print("Shape after target cleanup:", df.shape)

# =========================
# STEP 2 — Light cleaning
# =========================
# "36 months" -> 36
if "term" in df.columns and df["term"].dtype == "object":
    df["term"] = df["term"].astype(str).str.extract(r"(\d+)").astype(float)

# "13.99%" -> 13.99
if "int_rate" in df.columns and df["int_rate"].dtype == "object":
    df["int_rate"] = pd.to_numeric(df["int_rate"].astype(str).str.replace("%", "", regex=False), errors="coerce")

# drop redundant grade (sub_grade kept)
drop_optional = [c for c in ["grade"] if c in df.columns]
if drop_optional:
    print("Dropping optional redundant columns:", drop_optional)
    df = df.drop(columns=drop_optional)

# =========================
# STEP 3 — Drop leaky columns (post-origination / identifiers)
# =========================
LEAKY_COLS = [
    # outcome / payment history
    "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv",
    "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee",
    "last_pymnt_d", "last_pymnt_amnt", "last_credit_pull_d", "next_pymnt_d",

    # hardship / settlement / plans
    "hardship_flag", "hardship_type", "hardship_reason", "hardship_status",
    "hardship_amount", "hardship_start_date", "hardship_end_date",
    "hardship_length", "hardship_dpd", "hardship_loan_status",
    "payment_plan_start_date",
    "debt_settlement_flag", "debt_settlement_flag_date",
    "settlement_status", "settlement_date", "settlement_amount",
    "settlement_percentage", "settlement_term",

    # newly identified leakage / identifiers
    "last_fico_range_high", "last_fico_range_low",
    "url", "pymnt_plan", "id"
]
present_leaky = [c for c in LEAKY_COLS if c in df.columns]
df = df.drop(columns=present_leaky)
print(f"\nDropped {len(present_leaky)} leaky columns: {present_leaky}")

# =========================
# STEP 4 — Drop very-missing columns (>40% NaN)
# =========================
MISS_THRESH = 0.40
miss_ratio = df.isna().mean().sort_values(ascending=False)
to_drop = miss_ratio[miss_ratio > MISS_THRESH].index.tolist()
to_drop = [c for c in to_drop if c != "loan_status"]

if to_drop:
    print(f"\nColumns dropped for high missingness (> {int(MISS_THRESH*100)}%): {to_drop}")
    df = df.drop(columns=to_drop)

print("\nTop missing % after column drop:\n", (df.isna().mean().sort_values(ascending=False).head(15)*100).round(2))

# =========================
# STEP 5 — Downsample to ~500k rows (stratified)
# =========================
RSEED = 42
TARGET_ROWS = 500_000
if len(df) > TARGET_ROWS:
    frac_keep = TARGET_ROWS / len(df)
    print(f"\nDownsampling rows to ~{TARGET_ROWS} (keep fraction ≈ {frac_keep:.3f})...")
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1-frac_keep, random_state=RSEED)
    y_tmp = df["loan_status"]
    for keep_idx, _ in sss.split(df, y_tmp):
        df = df.iloc[keep_idx].copy()
    print("New shape after downsampling:", df.shape)
    print("New target %:", (df["loan_status"].value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 6 — Split
# =========================
y_full = df["loan_status"].astype("int8")
X_full = df.drop(columns=["loan_status"])

X_train, X_temp, y_train, y_temp = train_test_split(
    X_full, y_full, test_size=0.30, stratify=y_full, random_state=RSEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RSEED
)

print("\nSizes ->", X_train.shape, X_val.shape, X_test.shape)
print("Class % (train):", (y_train.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (val)  :", (y_val.value_counts(normalize=True)*100).round(2).to_dict())
print("Class % (test) :", (y_test.value_counts(normalize=True)*100).round(2).to_dict())

# =========================
# STEP 7 — Preprocessing (sparse OHE + rare bucket)
# =========================
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config

set_config(transform_output="default")

# Identify columns by dtype on X_full (after drops)
num_cols = [c for c in X_full.columns if pd.api.types.is_numeric_dtype(X_full[c])]
cat_cols = [c for c in X_full.columns if c not in num_cols]  # treat everything else as categorical

print(f"\nDetected {len(num_cols)} numeric and {len(cat_cols)} categorical columns.")

# Note: trees don't need scaling, but keeping it doesn't hurt; leaves numeric dense.
numeric_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True)),
    ("to_float32", FunctionTransformer(lambda X: X.astype(np.float32)))
])

categorical_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(lambda X: X.astype(str))),
    ("ohe", OneHotEncoder(
        handle_unknown="infrequent_if_exist",
        min_frequency=0.005,        # keep cats >=0.5% freq; others -> 'infrequent'
        sparse_output=True,
        dtype=np.float32,
        drop="if_binary"
    ))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=0.1
)

# =========================
# STEP 8 — Models (now incl. DecisionTree & RandomForest)
# =========================
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
try:
    from xgboost import XGBClassifier
except Exception as e:
    raise RuntimeError("xgboost not installed. Run: pip install xgboost") from e

scale_pos_weight = max(1.0, (y_train == 0).sum() / max(1, (y_train == 1).sum()))
print("scale_pos_weight (train):", round(scale_pos_weight, 3))

logit_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(
        solver="saga",
        penalty="l2",
        C=1.0,
        max_iter=300,
        class_weight="balanced",
        random_state=RSEED,
        n_jobs=-1
    ))
])

# Decision Tree (conservative to control memory/overfit)
dt_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", DecisionTreeClassifier(
        criterion="gini",
        max_depth=12,            # tune up/down as needed
        min_samples_leaf=50,     # larger leaf reduces overfitting and memory
        class_weight="balanced",
        random_state=RSEED
    ))
])

# Random Forest (sparse-friendly in recent sklearn; params conservative)
rf_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=14,            # tune as needed
        min_samples_leaf=20,
        max_features="sqrt",
        class_weight="balanced",
        n_jobs=-1,
        random_state=RSEED
    ))
])

xgb_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", XGBClassifier(
        n_estimators=250,
        max_depth=6,
        learning_rate=0.08,
        subsample=0.9,
        colsample_bytree=0.8,
        min_child_weight=5,
        reg_lambda=1.0,
        objective="binary:logistic",
        scale_pos_weight=scale_pos_weight,
        tree_method="hist",
        random_state=RSEED,
        n_jobs=-1,
        max_bin=256
    ))
])

models = {
    "LogReg": logit_pipe,
    "DecisionTree": dt_pipe,
    "RandomForest": rf_pipe,
    "XGBoost": xgb_pipe
}
print("Models ready:", list(models.keys()))

# =========================
# STEP 9 — Evaluation (Precision @ target recall)
# =========================
RECALL_TARGETS = (0.80, 0.85, 0.90)

def pick_threshold_at_min_recall(y_true, p1, min_recall):
    """Pick threshold that achieves recall >= min_recall with max precision (on validation)."""
    prec, rec, ths = precision_recall_curve(y_true, p1, pos_label=1)
    idxs = np.where(rec >= min_recall)[0]
    if len(idxs) == 0:
        return None, None, None
    best_i = idxs[np.argmax(prec[idxs])]
    thr = ths[min(best_i, len(ths)-1)]   # ths has len = len(prec)-1
    return float(thr), float(prec[best_i]), float(rec[best_i])

def eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test, recall_targets=RECALL_TARGETS):
    pipe.fit(X_train, y_train)

    # Validation for threshold selection
    p_val = pipe.predict_proba(X_val)[:, 1]
    ap = average_precision_score(y_val, p_val)
    print(f"\n=== {name} ===  AP(Val)={ap:.4f}")

    results = {}
    for tgt in recall_targets:
        thr, p_at, r_at = pick_threshold_at_min_recall(y_val, p_val, tgt)
        if thr is None:
            print(f"  recall≥{tgt:.2f}: not reachable.")
            continue

        p_test = pipe.predict_proba(X_test)[:, 1]
        yhat = (p_test >= thr).astype("int8")
        cm = confusion_matrix(y_test, yhat, labels=[1, 0])  # rows: actual 1,0 ; cols: pred 1,0
        TP, FN, FP, TN = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
        prec_test = TP/(TP+FP) if (TP+FP) > 0 else 0.0
        rec_test  = TP/(TP+FN) if (TP+FN) > 0 else 0.0

        print(f"  recall≥{tgt:.2f}: thr={thr:.4f} | Val P={p_at:.3f}, R={r_at:.3f} | Test precision={prec_test:.3f}, recall={rec_test:.3f}")
        results[tgt] = {"thr": thr, "precision_test": prec_test, "recall_test": rec_test, "cm": cm}
    return ap, results

all_results = {}
for name, pipe in models.items():
    ap, res = eval_model(name, pipe, X_train, y_train, X_val, y_val, X_test, y_test)
    all_results[name] = {"AP_val": ap, "by_recall": res}

print("\nSummary (precision at target recalls on TEST):")
for name, info in all_results.items():
    row = [name]
    for tgt in RECALL_TARGETS:
        if tgt in info["by_recall"]:
            row.append(f"R≥{tgt:.2f}: P={info['by_recall'][tgt]['precision_test']:.3f}")
        else:
            row.append(f"R≥{tgt:.2f}: NA")
    print(" | ".join(row))


Raw shape: (2260701, 151)
         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  installment grade sub_grade     emp_title emp_length home_ownership  annual_inc  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months     13.99       123.03     C        C4       leadman  10+ years       MORTGAGE     55000.0   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months     11.99       820.28     C        C1      Engineer  10+ years       MORTGAGE     65000.0   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months     10.78       432.66     B        B4  truck driver  10+ years       MORTGAGE     63000.0   

  verification_status   issue_d loan_status pymnt_plan                                                url desc             purpose               title zip_code addr_state    dti  \
0        Not Verified  Dec-2015  Fully Paid          n  https://lendingclub.com/browse/loanDetail.acti...  NaN  

Dropped 915391 rows with invalid loan_status values.

Target distribution (%):
loan_status
fully_paid(0)    80.04
default(1)       19.96
Name: proportion, dtype: float64
Shape after target cleanup: (1345310, 151)
Dropping optional redundant columns: ['grade']

Dropped 36 leaky columns: ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'next_pymnt_d', 'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'payment_plan_start_date', 'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term', 'last_fico_range_high', 'last_fico_range_low', 'url', 'pymnt_plan', 'id']

Columns dropped for hi